<a href="https://colab.research.google.com/github/Sebato/MLProject/blob/main/projetYES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importation des différentes librairies utiles pour le notebook

!pip install langdetect 
!pip install contractions
!pip install wordcloud

#Sickit learn met régulièrement à jour des versions et 
#indique des futurs warnings. 
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# librairies générales
import pickle 
import pandas as pd
from scipy.stats import randint
import numpy as np
import string
import time
import base64
import re
import sys

import contractions

# librairie BeautifulSoup
from bs4 import BeautifulSoup

# librairie affichage
import matplotlib.pyplot as plt
import seaborn as sns
import wordcloud

## detection de language
import langdetect 

import nltk
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from nltk import RegexpParser
# il est possible de charger l'ensemble des librairies en une seule fois 
# décocher le commentaire de la ligne ci-dessous
#nltk.download('all') 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag
nltk.download('tagsets')
nltk.download("stopwords")
nltk.download('wordnet')

from nltk.corpus import stopwords

import spacy
from spacy.tokens import Span
# il faut sélectionner pour quelle langue les traitements vont être faits.




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=6e0cc18782b86d26161ec4c60094819919056dc0890e51635c995362d7f9e2de
  Stored in directory: /root/.cache/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 KB 14.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


#Connexion au GitHub

In [ ]:
try: 
  projectDir
except NameError:
    %cd /content
    !git clone git@github.com:Sebato/MLProject.git
    projectDir = "/content/MLProject"
    sys.path.append(projectDir)
else:
  %cd /content/MLProject
  !git pull

# Se positionner sur le répertoire associé
%cd $projectDir

/content/MLProject
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 179.40 KiB | 4.08 MiB/s, done.
From https://github.com/Sebato/MLProject
   448be42..a4d2822  main       -> origin/main
Updating 448be42..a4d2822
Fast-forward
 mis_en_forme.ipynb | 893 ++++++++++++++++++++++++++++++++++-------------------
 1 file changed, 583 insertions(+), 310 deletions(-)
/content/MLProject


In [ ]:
nltk.download("stopwords")
from nltk.corpus import stopwords
import nltk
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/e20220002252/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/e20220002252/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
import pandas as pd
df = pd.read_csv('data/HAI817_Projet_train.csv')
df.drop_duplicates(subset=['title'], keep='first', inplace = True)
display(df)

,public_id,text,title,our rating
0,5a228e0e,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false
1,30c605a1,Missouri politicians have made statements afte...,Missouri lawmakers condemn Las Vegas shooting,mixture
2,c3dea290,Home Alone 2: Lost in New York is full of viol...,CBC Cuts Donald Trump's 'Home Alone 2' Cameo O...,mixture
3,f14e8eb6,But things took a turn for the worse when riot...,Obama’s Daughters Caught on Camera Burning US ...,false
4,faf024d6,It’s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff’s 78 Visits ...,false
...,...,...,...,...
1248,13aaf021,A blizzard! The hottest year ever! More signs ...,The Climate Snow Job,mixture
1250,6c425155,Increased atmospheric carbon dioxide has helpe...,The Phony War Against CO2\n,mixture
1251,f0669fad,Considering the importance of immunity boostin...,"Government of India Ministry of Ayurveda, Yoga...",false
1252,8060b507,"Even if world temperatures rise, the appropria...",Climate Change Isn’t the End of the World,mixture


In [ ]:
data = {
    "A": ["TeamA", "TeamB", "TeamB", "TeamC", "TeamA"],
    "B": [50, 40, 40, 30, 50],
    "C": [True, False, False, False, True]
}
  
df = pd.DataFrame(data)
display(df)
display(df.drop_duplicates())

,A,B,C
0,TeamA,50,True
1,TeamB,40,False
2,TeamB,40,False
3,TeamC,30,False
4,TeamA,50,True


,A,B,C
0,TeamA,50,True
1,TeamB,40,False
3,TeamC,30,False


In [ ]:
display(dd)

,public_id,text,title,our rating
0,5a228e0e,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false
1,30c605a1,Missouri politicians have made statements afte...,Missouri lawmakers condemn Las Vegas shooting,mixture
2,c3dea290,Home Alone 2: Lost in New York is full of viol...,CBC Cuts Donald Trump's 'Home Alone 2' Cameo O...,mixture
3,f14e8eb6,But things took a turn for the worse when riot...,Obama’s Daughters Caught on Camera Burning US ...,false
4,faf024d6,It’s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff’s 78 Visits ...,false
...,...,...,...,...
1248,13aaf021,A blizzard! The hottest year ever! More signs ...,The Climate Snow Job,mixture
1250,6c425155,Increased atmospheric carbon dioxide has helpe...,The Phony War Against CO2\n,mixture
1251,f0669fad,Considering the importance of immunity boostin...,"Government of India Ministry of Ayurveda, Yoga...",false
1252,8060b507,"Even if world temperatures rise, the appropria...",Climate Change Isn’t the End of the World,mixture


# Préparation des données
- Tokenization et mise en lowercase
### Création de liste des mots qui apparaissent dans le texte et dans le titre de chaque document 

In [ ]:
def cleaningData(data):
 """etapes de nettoyage:
  - supprimer doublons
  - suppression de la fin des URLs
  - transformation des caractères numériques en mots
  - 
      
  """
  # regex = re.compile('(?u)\\b\\w\\w+\\b')
  # tokens = re.findall(regex, text.lower()) # notice lowercase=True param
  # vocab = sorted(set(tokens))
  # counts = Counter(tokens)
  # counts = [counts[key] for key in sorted(counts.keys())]
  # #vocab, counts = list(zip(*sorted(counter.items()))) # one liner with asterisk unpacking
  # print(pd.DataFrame([counts], columns = vocab))


In [ ]:
import re


#Prise en compte des éléments qui doivent être regroupés
regex_str = [
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # nombres
    r"(?:[a-z][a-z'\-_]+[a-z])", # mots avec - et '
    r'(?:[\w_]+)', # autres mots
    r'(?:\S)' # le reste
]

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)

def tokenize(s):
  tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
  
  return tokens_re.findall(s)

tokenPattern = r'('+'|'.join(regex_str)+')'
print("\n--------\n"+tokenPattern)
    
def preprocess(text):
  """ etapes de pre processing :
      - suppréssion de la fin des URLs
      - transformation des caractères numériques en mots
      - 
      
  """
  return text.lower()
  


--------
(<[^>]+>|(?:@[\w_]+)|(?:\#+[\w_]+[\w\'_\-]*[\w_]+)|http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+|(?:(?:\d+,?)+(?:\.?\d+)?)|(?:[a-z][a-z'\-_]+[a-z])|(?:[\w_]+)|(?:\S))


In [ ]:
def get_tokens(dataframe):    
    full_texts = []
    for i in range(len(dataframe)):
        full_texts.append((str(dataframe.iloc[i].title) + " " + str(dataframe.iloc[i].text)))
    tokens = [t.split() for t in full_texts]
    tokens = [[t.lower() for t in lt] for lt in tokens]
    return tokens


In [ ]:
def elim_stopwords(words, stopwords):
    return [word for word in words if word not in stopwords]

    

In [ ]:
def stemming(dataframe):
    """ """
    ps=nltk.stem.porter.PorterStemmer()
    dataframe["tokens"].transform(lambda tokens: [ps.stem(word) for word in tokens])
    return dataframe


In [ ]:
def prepare_data(dataframe, classes):
    dataframe["tokens"] = get_tokens(dataframe)
    
    swords = set(stopwords.words("english"))
    dataframe["tokens"] = dataframe["tokens"].transform(lambda t: [word for word in t if word not in swords])
    
    #retirer émoticones
    

    ps=nltk.stem.porter.PorterStemmer()
    dataframe["tokens"] = dataframe["tokens"].transform(lambda tokens: [ps.stem(word) for word in tokens])

    dataframe = dataframe[dataframe["our rating"].isin(classes)]
    dataframe = dataframe.reset_index(drop=True)
        
    return dataframe

In [ ]:
df_train = prepare_data(df, ["true", "false"])


In [ ]:
(display(df_train.head()))

,public_id,text,title,our rating,langue,tokens
0,5a228e0e,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false,en,"[fine, $1,500, passeng, use, mobil, phone,, st..."
1,f14e8eb6,But things took a turn for the worse when riot...,Obama’s Daughters Caught on Camera Burning US ...,false,en,"[obama’, daughter, caught, camera, burn, us, f..."
2,faf024d6,It’s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff’s 78 Visits ...,false,en,"[leak, visitor, log, reveal, schiff’, 78, visi..."
3,61bd9a69,"November 23, 2019 The U.S. Food and Drug Admi...",FDA Shocking Study: Cells Used In Vaccines Con...,false,en,"[fda, shock, study:, cell, use, vaccin, contam..."
4,bb1999cc,"Trump confirms this was a bombing, not an acci...","Israel Hits Beirut with Nuclear Missile, Trump...",false,en,"[israel, hit, beirut, nuclear, missile,, trump..."


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def create_bag(dataframe):
    corpus = []
    for i in range(len(dataframe)):
        corpus.append((str(dataframe.iloc[i].title) + " " + str(dataframe.iloc[i].text)))
    vectorizer = CountVectorizer(preprocessor=preprocess, token_pattern = tokenPattern, stop_words = list(set(stopwords.words("english"))))
    # creation du vocabulaire
    vector = vectorizer.fit_transform(corpus)
    df = pd.DataFrame(data=vectorizer.transform(corpus).toarray(), columns=vectorizer.get_feature_names_out())
    df.insert(0, "our_rating", dataframe["our rating"])
    return (vectorizer, df)
    


In [ ]:
bag, bag_train = create_bag(df_train)
print(bag.vocabulary_)


{'you': 28221, 'can': 5132, 'be': 3885, 'fined': 10664, '$': 72, '1,500': 178, 'if': 13296, 'your': 28234, 'passenger': 18971, 'is': 14222, 'using': 26946, 'a': 1850, 'mobile': 16865, 'phone': 19387, ',': 80, 'starting': 24249, 'next': 17653, 'week': 27634, 'distracted': 8637, 'driving': 8949, 'causes': 5415, 'more': 17029, 'deaths': 7682, 'in': 13498, 'canada': 5134, 'than': 25486, 'impaired': 13411, '.': 82, 'it': 14271, '’': 28368, 's': 22374, 'why': 27804, 'every': 9942, 'province': 20526, 'and': 2758, 'territory': 25431, 'has': 12322, 'laws': 15123, 'against': 2327, 'while': 27761, 'operating': 18356, 'cell': 5471, '“': 28369, 'tell': 25346, 'passengers': 18972, 'to': 25798, 'stay': 24308, 'off': 18181, 'their': 25517, 'phones': 19389, 'are': 3094, 'such': 24707, 'measures': 16318, 'necessary': 17510, 'now': 17957, 'as': 3198, 'claimed': 5906, 'lives': 15533, 'provinces': 20527, 'like': 15426, 'british': 4731, 'columbia': 6265, 'ontario': 18329, 'quebec': 20762, 'alberta': 2462, '

In [ ]:
print([col for col in bag_train.columns if 'you' in col])

['bayou', 'http://www.ritholtz.com/blog/2012/02/is-your-state-a-net-giver-or-taker-of-federal-taxes/', 'https://www.youtube.com/watch?v=eknc0j3xx1w&feature=youtu.be&t=156', 'https://www.youtube.com/watch?v=f4_dnabtptu', 'https://www.youtube.com/watch?v=lnzotrazjzg', 'https://www.youtube.com/watch?v=omxfffqydtc&feature=youtu.be&t=173', 'https://www.youtube.com/watch?v=vt57ep1wl9g&feature=share', 'https://youtu.be/iqjvrcsejwu', 'https://youtu.be/ywfxngnvvuu', 'make-your-own-trump-hat', 'payouts', 'worse-than-you-could-have-ever-imagined', 'you-know-what', 'you-know-who', 'young', 'young-onset', 'younger', 'youngest', 'youngster', 'youngsters', 'youre', 'youth', 'youthimage', 'youths', 'youtube']


In [ ]:

bag_train.head()

,our_rating,!,"""",#,#16,#abes,#americafirst,#antarctica,#arctic,#asuu,...,,️,￼,🆘,🇬,🇳,🎙,🎥,😷,🤔
0,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,false,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,false,2,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
display(bag_train)

,our_rating,00,000,0008,000m,000s,000x,001,002,003,...,zucker,zuckerburg,zundel,zurich,zwar,zxc21,zzt2briays,ärzteblatt,único,μm
0,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784,true,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
785,true,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
786,false,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
787,true,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
bag_train.to_csv("bow.csv")

KeyError: ignored

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [ ]:
X = bag_train.values[:, 1:]
y = bag_train.values[:, 0]

models = []
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVC', SVC(gamma='auto')))
models.append(('LSVC', LinearSVC()))
models.append(('RFC', RandomForestClassifier()))

seed = 7
results = []
names = []
scoring='accuracy'
for name,model in models:
    kfold = KFold(n_splits=10, random_state=seed,shuffle=True)
    cv_results = cross_val_score(model, X, y, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg) 

/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

LR: 0.831435 (0.040826)
KNN: 0.762999 (0.051546)
NB: 0.783268 (0.048845)
SVC: 0.749075 (0.059213)


/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1244

KeyboardInterrupt: 

In [ ]:
clf = LogisticRegression()
clf.fit(X, y)
res = clf.score(X, y)
print(res)

/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9987325728770595
